## Criando uma RAG

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini")

In [9]:
caminhos = [
    "../pdfes/Explorando a API da OpenAI.pdf",
    "../pdfes/Explorando o Universo das IAs com Hugging Face.pdf",
    "../pdfes/Miyamoto Musashi - Gorin no Sho - O Livro dos Cinco Elementos.pdf",
    "../pdfes/ultra-aprendizado-ultralearning-scott-young.pdf"
]

In [10]:
paginas = []
for pdf in caminhos:
    loader = PyPDFLoader(pdf)
    paginas.extend(loader.load_and_split())

print(len(paginas))

551


In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,
    separators=[".", "!","\n\n"]
)

chunks = text_splitter.split_documents(paginas)

def limpa_texto(texto):
    return texto.replace("\n", " ").strip()

for chunk in chunks:
    chunk.page_content = limpa_texto(chunk.page_content)

print(chunks[500].page_content)

.      Esmagar  Essa é a tática aplicável quando se  considera que o inimigo é fraco.  Julgando-se mais forte, você toma a de cisão de esmagá-lo sem qualquer perda  de tempo.  No caso da arte militar aplicada a combate de exércitos, quando se  percebe que o número de adversár ios é pequeno — ou mesmo numeroso,  mas de ânimo vacilante e nervoso —, tr ate de esmagá-los completamente com  todas as suas forças. Se o ímpeto de esmagar for fraco, os adversários se  recuperarão. É preciso ter clara na ment e a idéia de esmagá-los na palma da  mão.  O mesmo deve ocorrer na arte militar de combate individual. Se cr  inimigo for mais fraco, se notar perturba ção em seu ritmo, ou se perceber que  ele está recuando, é essencial acab ar com ele sem perda de tempo. O  importante é não lhe dar a chance de recobrar o fôlego.  Examinar bem o assunto


In [30]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(
    chunks,
    embedding=embeddings
)

In [31]:
retriever = vectorstore.as_retriever(
    search_type="mmr", 
    search_kwargs={"k": 5}
)

retriever.invoke("Como ser um samurai?")

[Document(id='d3b4989d-79d8-4352-a6b4-fc99c1918e02', metadata={'source': '../pdfes/Miyamoto Musashi - Gorin no Sho - O Livro dos Cinco Elementos.pdf', 'page': 45}, page_content='É preciso ponderar com calma e to mar como missão de samurai a  prática desse método.  Hoje você terá a vitória sobre o que  foi ontem; amanhã, triunfará sobre  os menos preparados; depois, sobre os mais competentes.  Seguindo o exposto neste livro, este ja sempre atento para nunca se  desviar dos verdadeiros mandamentos. Mesmo que vença alguns inimigos, se  isso for feito contrariando as normas en sinadas aqui, você estará fugindo dos  verdadeiros mandamentos. Com o espírito impregnado dessas vantagens,  estará preparado para vencer até dezenas de antagonistas. Então, terá  aprendido a sabedoria do kenjutsu e, com ela, a arte militar aplicada ao  combate individual ou ao confronto de exércitos. É preciso forjar a sua arte da  espada com treinamentos de mil dias; depois, poli-la com treinos de dez mil  dias.  E

## Criando uma Chain



In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = ChatPromptTemplate.from_template(
    """Você é um assistente de IA que responde perguntas de acordo com o contexto fornecido.

    Contexto: {contexto}
    Pergunta: {pergunta}
    """
)
output_parser = StrOutputParser()

In [35]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup_and_retriever = RunnableParallel({
    "pergunta": RunnablePassthrough(),
    "contexto": retriever
})

chain = setup_and_retriever | template | model | output_parser

chain.invoke("Como ser um samurai?")

'Para ser um samurai, é fundamental seguir um caminho que envolve tanto a prática das artes marciais quanto o desenvolvimento intelectual e espiritual. Aqui estão alguns pontos importantes a considerar:\n\n1. **Prática das Artes Marciais**: O samurai deve dedicar-se ao treinamento rigoroso, como mencionado no contexto, que inclui mil dias de treinamento seguidos de dez mil dias de prática para polir suas habilidades na arte da espada. Isso se refere ao kenjutsu, a arte da espada, que é essencial para um guerreiro.\n\n2. **Conhecimento dos Mandamentos**: É crucial conhecer e seguir os mandamentos da arte militar, que envolvem não apenas a luta, mas uma compreensão mais profunda da disciplina, ética e filosofia do samurai. Isso inclui o aprendizado de outras áreas como poesia, cerimônias e medicina, refletindo a ideia de "Bun-bu Nidô", que combina o caminho das letras e das artes marciais.\n\n3. **Espírito e Filosofia**: O samurai deve ter um espírito que valoriza a vitória sobre si mesm

#### Entendendo a Chain


In [38]:
RunnablePassthrough().invoke("Como ser um samurai?") # Passa a pergunta para o modelo

retriever.invoke("Como ser um samurai?") # Passa a pergunta para o retriever

RunnableParallel({  #É uma função que recebe um dicionário de inputs e retorna um dicionário de outputs resumidamente
    "pergunta": RunnablePassthrough(),
    "contexto": retriever
}).invoke("Como ser um samurai?") # Passa a pergunta para o retriever e retorna o contexto

{'pergunta': 'Como ser um samurai?',
 'contexto': [Document(id='d3b4989d-79d8-4352-a6b4-fc99c1918e02', metadata={'source': '../pdfes/Miyamoto Musashi - Gorin no Sho - O Livro dos Cinco Elementos.pdf', 'page': 45}, page_content='É preciso ponderar com calma e to mar como missão de samurai a  prática desse método.  Hoje você terá a vitória sobre o que  foi ontem; amanhã, triunfará sobre  os menos preparados; depois, sobre os mais competentes.  Seguindo o exposto neste livro, este ja sempre atento para nunca se  desviar dos verdadeiros mandamentos. Mesmo que vença alguns inimigos, se  isso for feito contrariando as normas en sinadas aqui, você estará fugindo dos  verdadeiros mandamentos. Com o espírito impregnado dessas vantagens,  estará preparado para vencer até dezenas de antagonistas. Então, terá  aprendido a sabedoria do kenjutsu e, com ela, a arte militar aplicada ao  combate individual ou ao confronto de exércitos. É preciso forjar a sua arte da  espada com treinamentos de mil dias

#### Criando uma alternativa não paralelizavel




In [39]:
setup_dict = {
    "pergunta": RunnablePassthrough(), 
    "contexto": retriever
}
chain = setup_dict | template | model | output_parser
chain.invoke("Como ser um samurai?")

'Para ser um samurai, segundo os ensinamentos de Miyamoto Musashi, é necessário seguir um caminho que envolve tanto o domínio das artes marciais quanto o aprendizado das letras. Aqui estão alguns pontos cruciais para se tornar um samurai:\n\n1. **Domínio das Artes Marciais**: É essencial praticar e aperfeiçoar as habilidades de combate, especialmente no kenjutsu (arte da espada). Musashi menciona que é necessário treinar por mil dias para forjar sua arte da espada e depois polir essa habilidade com dez mil dias de prática.\n\n2. **Conhecimento das Letras**: O samurai deve também se dedicar ao aprendizado cultural, incluindo a apreciação da poesia, da cerimônia do chá e de outras artes. Isso reflete o princípio do Bun-bu Nidō, que enfatiza a importância de equilibrar as artes marciais com o conhecimento intelectual.\n\n3. **Seguir os Mandamentos**: É fundamental conhecer e respeitar os mandamentos da arte militar, que orientam o samurai em sua conduta e decisões. Musashi alerta que mesm

#### Utilizando Fallback

O fallback é um mecanismo de segurança que permite ter um "plano B" quando algo falha na execução da chain. Em outras palavras, é uma alternativa que entra em ação quando o componente principal não funciona como esperado.

No contexto do LangChain, você pode usar fallbacks para:
Modelos de linguagem (LLMs):
Se um modelo falhar ou ficar indisponível, outro modelo pode ser usado como backup
Por exemplo: se o GPT-4 falhar, pode usar o GPT-3.5 como fallback

Embeddings:
Se um serviço de embeddings falhar, outro pode ser usado
Por exemplo: se OpenAI Embeddings falhar, pode usar HuggingFace Embeddings

Retrievers:
Se um método de recuperação falhar, outro pode ser usado
Por exemplo: se a busca por similaridade falhar, pode usar busca por palavras-chave


In [49]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(model="gpt-3.5-turbo-instruct")
prompt = PromptTemplate.from_template("Resuma o seguinte texto: {texto}")

chain_pequena = prompt | llm
chain_pequena.invoke({"texto":"Oi, tudo bem?"})

'\n\nO texto é uma saudação informal perguntando se a pessoa está bem.'

In [50]:
chain_pequena.invoke({"texto":"Oi, tudo bem?" *1000})

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 5264 tokens (5008 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [58]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
prompt = ChatPromptTemplate.from_template("Resuma o seguinte texto: {texto}")

chain_grande = prompt | llm | StrOutputParser()
chain_grande.invoke({"texto":"Oi, tudo bem?" *1000})

'Oi, tudo bem? (repetido muitas vezes)'

In [56]:
chain_fallback = chain_pequena.with_fallbacks([chain_grande])
chain_fallback.invoke({"texto":"Oi, tudo bem?" *1000})

'Oi, tudo bem? (repetido várias vezes)'

#### Fallback de Formatação


In [66]:
from langchain_openai import OpenAI
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain.output_parsers import DatetimeOutputParser

prompt = PromptTemplate.from_template(
    "Qual foi a data do evento {evento} (no formato %Y-%m-%dT%H:%M:%S.%Z - Retorne apenas o valor solicitado)?"
)

chain_instruct = prompt | OpenAI(model="gpt-3.5-turbo-instruct") | DatetimeOutputParser()
chain_instruct.invoke({"evento":"Qual foi a data do final da copa de 2002?"})

datetime.datetime(2002, 6, 30, 0, 0)

In [64]:
chain_turbo = prompt | ChatOpenAI(model="gpt-3.5-turbo-0125") | DatetimeOutputParser()
chain_turbo.invoke({"evento":"Qual foi a data do final da copa de 2002?"})

datetime.datetime(2002, 6, 30, 0, 0)

In [67]:
chain_fallback = chain_instruct.with_fallbacks([chain_turbo])
chain_fallback.invoke({"evento":"Qual foi a data do final da copa de 2002?"})

datetime.datetime(2002, 6, 30, 0, 0)